In [15]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [16]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-125b  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [17]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar 12 00:17 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [19]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [20]:
spark

In [21]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '322514282' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        print(b.name)
        paths.append(full_path+b.name)

multistream10_preprocessed.parquet
multistream11_part2_preprocessed.parquet
multistream11_preprocessed.parquet
multistream12_part2_preprocessed.parquet
multistream12_preprocessed.parquet
multistream13_part2_preprocessed.parquet
multistream13_preprocessed.parquet
multistream14_part2_preprocessed.parquet
multistream14_preprocessed.parquet
multistream15_part2_preprocessed.parquet
multistream15_part3_preprocessed.parquet
multistream15_preprocessed.parquet
multistream16_part2_preprocessed.parquet
multistream16_part3_preprocessed.parquet
multistream16_preprocessed.parquet
multistream17_part2_preprocessed.parquet
multistream17_part3_preprocessed.parquet
multistream17_preprocessed.parquet
multistream18_part2_preprocessed.parquet
multistream18_part3_preprocessed.parquet
multistream18_preprocessed.parquet
multistream19_part2_preprocessed.parquet
multistream19_part3_preprocessed.parquet
multistream19_preprocessed.parquet
multistream1_preprocessed.parquet
multistream20_part2_preprocessed.parquet
m

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [22]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("id", "title").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [23]:
# Count number of wiki pages
parquetFile.count()

6348910

In [24]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [25]:
dt_dict = doc_title_pairs.collectAsMap()

In [26]:
dt_clean = 'dt.pkl'
with open(dt_clean, 'wb') as f:
    pickle.dump(dt_dict, f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"dt/{dt_clean}")

blob.upload_from_filename(dt_clean)

In [27]:
len(dt_dict)

6348910